In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashteamstats
import cv2
import matplotlib.image as image
fig_DIR = "../figs/team_leaders/"
box_DIR = "../data/box/"
img_DIR_T = "../data/images/teams/"
img_DIR_P = "../data/images/players/"


## Playoff Scoring Distribution

In [ ]:
dfa = []
for year in range(2003,2023):
    stats = leaguedashteamstats.LeagueDashTeamStats(
            measure_type_detailed_defense="Scoring",
            season_type_all_star="Playoffs",
            season=get_ss(year)
        )
    df1 = stats.get_data_frames()[0]
    df2 = df1.query("W_RANK  == 1")
    df2["Season"] = year + 1
    dfa.append(df2)
df3 = pd.concat(dfa)
df3 = df3.reset_index(drop=True)
df3["url"]= img_DIR_T + df3["TEAM_NAME"] + ".png"
# df3["image"]=df3["url"].apply(image.imread)

In [ ]:
yvar = "PCT_AST_2PM"
title = "Unassisted 2PT"
p = (
    ggplot(df3,aes(x="Season",y=yvar,image="url"))
    + geom_line(size=2)
    + geom_image(size=0.06)
    # + geom_smooth(se=False,color="red",size=2)
    # + geom_image(aes(x="Season",y=yvar,image="url"),size=0.06)
    + labs(
        title=f"NBA Champions Percentage of Points:{title}",
        # subtitle="red line: Trend line using Local regression",
        caption="@sradjoker | source:nba.com/stats",
        x="Season",
        y="Percentage of Points"
    )
    + scale_y_continuous(labels=percent_format())
    # + scale_x_continuous(breaks=range(2013,2024,2))
    + theme_idv
    + theme(
        plot_title=element_text(size=14),
        axis_title_y=element_blank(),
        figure_size=(5,4)
    )
)
p

In [ ]:
dfa = []
for year in range(2003,2023):
    stats = leaguedashteamstats.LeagueDashTeamStats(
            measure_type_detailed_defense="Scoring",
            season_type_all_star="Playoffs",
            season=get_ss(year)
        )
    df1 = stats.get_data_frames()[0]
    df2 = df1.query("W_RANK  == 1")
    stats = leaguedashteamstats.LeagueDashTeamStats(
            measure_type_detailed_defense="Scoring",
            season_type_all_star="Regular Season",
            season=get_ss(year)
        )
    df3 = stats.get_data_frames()[0]
    df4 = pd.merge(df2,df3,on=["TEAM_ID","TEAM_NAME"])
    df4["Season"] = year + 1
    dfa.append(df4)
df5 = pd.concat(dfa)
df5 = df5.reset_index(drop=True)
# df5["url"]= img_DIR_T + df5["TEAM_NAME"] + ".png"

In [ ]:
df6 = df5[['TEAM_NAME',"PCT_PTS_2PT_MR_x","PCT_PTS_2PT_MR_y","Season",'url']]
df6["url"]= img_DIR_T + df6["TEAM_NAME"] + ".png"

In [ ]:
df7 = pd.melt(df6, id_vars=["TEAM_NAME","Season","url"], value_vars=["PCT_PTS_2PT_MR_x","PCT_PTS_2PT_MR_y"])
df7["variable"] = df7["variable"].str.replace("PCT_PTS_2PT_MR_x","Playoffs")
df7["variable"] = df7["variable"].str.replace("PCT_PTS_2PT_MR_y","Regular Season")
df7 = df7.rename(columns={"variable":"Segment"})

In [ ]:
kwargs_legend = {"alpha":0.0}
p = (
    ggplot(df7,aes(x="Season",y="value",color="Segment",image="url"))
    + geom_point(size=2)
    # + geom_line(size=2)
    # + geom_image(size=0.06)
    + geom_smooth(se=False,size=2)
    # + geom_image(aes(x="Season",y=yvar,image="url"),size=0.06)
    + labs(
        title=f"NBA Champions Percentage of Points: Mid Range Shots",
        # subtitle="red line: Trend line using Local regression",
        caption="@sradjoker | source:nba.com/stats",
        x="Season",
        y="Percentage of Points"
    )
    + scale_y_continuous(labels=percent_format())
    + scale_color_manual(values=["blue","red"])
    + scale_x_continuous(breaks=range(2004,2024,2))
    + theme_idv
    + theme(
        plot_title=element_text(size=16),
        axis_title_y=element_blank(),
        # figure_size=(6,4)
    )
    + theme(
        legend_title=element_blank(),
        legend_position = [0.8,0.85],
        legend_direction = "vertical",
        legend_box_margin=0,
        legend_background=element_rect(color="grey", size=0.001,**kwargs_legend), # type: ignore
        legend_box_background = element_blank(),
        legend_text=element_text(size=12),
    )
)
p

## Playoffs vs Regular Season

In [ ]:
cols1 = ['TEAM_NAME', 'GP', 'W', 'L', 'W_PCT',
       'OFF_RATING', 'DEF_RATING', 'NET_RATING', 'EFG_PCT', 'TS_PCT', 'E_PACE', 'PACE', 'POSS',
       "W_RANK", "L_RANK",
       'W_PCT_RANK', 'OFF_RATING_RANK', 'DEF_RATING_RANK',
       'NET_RATING_RANK',]

In [ ]:
dfa = []
dfb = []
for year in range(2003,2023):
    stats = leaguedashteamstats.LeagueDashTeamStats(
        measure_type_detailed_defense="Advanced",
        season = get_ss(year)
    )
    df1 = stats.get_data_frames()[0]
    df1 = df1.sort_values("W_PCT",ascending=False).reset_index(drop=True)
    stats = leaguedashteamstats.LeagueDashTeamStats(
        measure_type_detailed_defense="Advanced",
        season_type_all_star="Playoffs",
        season=get_ss(year)
    )
    df2 = stats.get_data_frames()[0]
    df2 = df2.sort_values("W",ascending=False).reset_index(drop=True)
    df1 = df1[cols1]
    df2 = df2[cols1]
    df3 = pd.merge(df2,df1,on=["TEAM_NAME"])
    df3["Season"] = year + 1
    stats = leaguegamelog.LeagueGameLog(
        player_or_team_abbreviation="T",
        season=get_ss(year),
        season_type_all_star="Playoffs",
    )
    df4 = stats.get_data_frames()[0]
    df4["Round_x"] = df4["GAME_ID"].str[-3]
    df4["Win"] = df4["WL"] == "W"
    df4["Loss"] = df4["WL"] == "L"
    df5 = pd.merge(df3,df4,on="TEAM_NAME")
    dfa.append(df3)
    dfb.append(df5)
df6 = pd.concat(dfa)
df7 = pd.concat(dfb)

In [ ]:
df6.columns

In [ ]:
xvar = "OFF_RATING_y"
yvar = "OFF_RATING_x"

r2 = (df6[xvar].corr(df6[yvar]))**2
p = (
    ggplot(df6,aes(x=xvar,y=yvar))
    + geom_point(color="red")
    # + geom_smooth(method="lm",color="red",size=2)
    + geom_abline(intercept = 0, slope = 1, color = "black",size = 2,linetype="--")
    + labs(
        title="Regular Season vs Playoffs: Offensive Rating",
        subtitle="Last 20 Seasons: From 2004-2023 | Each Dot is a Playoff Team\nTeams above black line have better playoff offenses",
        caption="@sradjoker | source:nba.com/stats",
        x="Regular Season",
        y="Playoffs"
    )
    + annotate('label', x=102, y=120, label=f'r^2={r2:0.3f}', size=14)
    + theme_idv
    + theme(
        figure_size=(5*1.1,4*1.1)
    )
)
p

In [ ]:
xvar = "OFF_RATING_y"
yvar = "OFF_RATING_x"
fig = px.scatter(df6, x=xvar, y=yvar,width=800, height=600, hover_name="TEAM_NAME",hover_data=["Season"],color_discrete_sequence =["red"])
# fig_line = px.line(df, x = "RAPM", y = "RAPM",color_discrete_sequence=["black"])
# fig = go.Figure(data = fig_10.data + fig_line.data)
fig.update_layout( width=800, height=600, showlegend=True)
fig.update_layout( title=dict(
                        text="Regular Season vs Playoffs: Offensive Rating",
                        xanchor="auto",
                        yanchor="middle",
                        font=dict(family="Arial, Tahoma, Helvetica", size=25, color="Black")
                    ),
            )
fig.update_xaxes(title_text=xvar)
fig.update_yaxes(title_text=yvar)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()

In [ ]:
yvar1 = "OFF_RATING_y"
yvar2 = "OFF_RATING_x"
kwargs_legend = {"alpha":0.0}
p = (
    ggplot(df6)
    + geom_smooth(aes(x="Season",y=yvar1,color="'blue'"),size=2)
    + geom_smooth(aes(x="Season",y=yvar2,color="'red'"),size=2)
    + scale_color_identity(guide='legend', labels=['Regular Season','Playoffs'])
    + labs(
        title="Regular Season vs Playoffs: Offensive Rating",
        # subtitle="Last 20 Seasons: From 2004-2023 | Each Dot is a Playoff Team\nTeams above black line have better playoff offenses",
        # caption="@sradjoker | source:nba.com/stats",
        x="Season",
        y="Offensive Rating"
    )
    + theme_idv
    + theme(
        figure_size=(5*1.1,4*1.1),
    )
    + theme(
        legend_title=element_blank(),
        legend_position = [0.3,0.85],
        legend_direction = "vertical",
        legend_box_margin=0,
        legend_background=element_rect(color="grey", size=0.001,**kwargs_legend), # type: ignore
        legend_box_background = element_blank(),
    )
)
p

In [ ]:
# df5 = df4.groupby(["TEAM_NAME","Round"])[["Win","Loss"]].sum().reset_index()
# df6 = df5.groupby(["TEAM_NAME"])[["Round"]].count().reset_index()

## Player Regular Season vs Playoffs

In [ ]:
from nba_api.stats.endpoints import leaguedashplayerstats

In [ ]:
dfa = []
for year in range(2003,2023):
    stats = leaguedashplayerstats.LeagueDashPlayerStats(
        measure_type_detailed_defense="Base",
        per_mode_detailed="PerGame",
        season_type_all_star="Regular Season",
        season=get_ss(year),
    )
    df1 = stats.get_data_frames()[0]
    stats = leaguedashplayerstats.LeagueDashPlayerStats(
        measure_type_detailed_defense="Base",
        per_mode_detailed="PerGame",
        season_type_all_star="Playoffs",
        season=get_ss(year),
    )
    df2 = stats.get_data_frames()[0]
    df3 = pd.merge(df2,df1,on=["PLAYER_ID","PLAYER_NAME"])
    df4 = df3.query("GP_x > 3 & GP_y > 3 & MIN_x >= 15 & MIN_y >= 15")
    df4["Season"] = year + 1
    dfa.append(df4)
df5 = pd.concat(dfa)
df5["image"] = img_DIR_P + df5["PLAYER_ID"].astype(str) + ".png"

In [ ]:
df6 = df5.query("PTS_y >= 25 & Season > 2012")
len(df6)

In [ ]:
data = df6
xvar = "PTS_y"
yvar = "PTS_x"
r2 = (data[xvar].corr(data[yvar]))**2
adjust_text_dict = {
    # 'expand_points': (0, 0),
    'arrowprops': {
        'arrowstyle': '->',
        'color': 'red'
    }
}
p = (
    ggplot(data,aes(x=xvar,y=yvar,image="image"))
    + geom_point(color="blue")
    # + geom_image(size=0.15)
    # + geom_smooth(method="lm",color="red",size=2)
    # + geom_text(aes(y = data[yvar] -0.5,label = "Season"),adjust_text=adjust_text_dict,size=8)
    + geom_text(aes(y = data[yvar],label = data["Season"].astype(str) + " " + data["PLAYER_NAME"]),adjust_text=adjust_text_dict,size=8)
    + geom_abline(intercept = 0, slope = 1, color = "black",size = 2,linetype="--")
    + labs(
        title="Regular Season vs Playoffs: Points",
        subtitle="Last 10 Seasons | Each Dot is a Playoff Player | Min 25 Pts Per Game\nPlayers above black line have better playoff numbers",
        caption="@sradjoker | source:nba.com/stats",
        x="Regular Season",
        y="Playoffs"
    )
    + annotate('label', x=35, y=16, label=f'r^2={r2:0.3f}', size=14)
    + theme_xkcd(base_size=16,stroke_size=1)
    + theme(
        plot_title=element_text(size=24),
        figure_size=(10,8),
    )
)
p.draw()

In [ ]:
xvar = "FG3M_y"
yvar = "FG3M_x"
fig = px.scatter(df5, x=xvar, y=yvar,width=800, height=600, hover_name="PLAYER_NAME",hover_data=["Season"],color_discrete_sequence =["red"])
# fig_line = px.line(df, x = "RAPM", y = "RAPM",color_discrete_sequence=["black"])
# fig = go.Figure(data = fig_10.data + fig_line.data)
fig.update_layout( width=800, height=600, showlegend=True)
fig.update_layout( title=dict(
                        text="Regular Season vs Playoffs: Offensive Rating",
                        xanchor="auto",
                        yanchor="middle",
                        font=dict(family="Arial, Tahoma, Helvetica", size=25, color="Black")
                    ),
            )
fig.update_xaxes(title_text=xvar)
fig.update_yaxes(title_text=yvar)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()